<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment_Phi_3.5_MoE_A100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment on Phi 3.5

* https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/discover-the-new-multi-lingual-high-quality-phi-3-5-slms/ba-p/4225280
* https://huggingface.co/microsoft/Phi-3.5-mini-instruct
* https://huggingface.co/microsoft/Phi-3.5-MoE-instruct

In [1]:
%%time

!pip install --upgrade -q transformers accelerate flash_attn torch bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 691.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.4.0 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, bu

In [2]:
import transformers
transformers.__version__


'4.44.2'

In [3]:
import accelerate
accelerate.__version__

'0.33.0'

In [4]:
import flash_attn
flash_attn.__version__

'2.6.3'

In [5]:
import torch
torch.__version__

'2.4.0+cu121'

In [6]:
import bitsandbytes
bitsandbytes.__version__

'0.43.3'

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
from IPython.display import Markdown

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in ca

In [10]:
!nvidia-smi

Fri Aug 23 07:16:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              47W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [11]:
# model_id = "microsoft/Phi-3-mini-4k-instruct"
model_id = "microsoft/Phi-3.5-MoE-instruct"

In [48]:
kind = 'Phi-3.5-MoE_4bit'

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [13]:
%%time

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config, # quantized
    # torch_dtype=torch.bfloat16,  # 16-Bit full resolution
    device_map="cuda",
    # https://huggingface.co/microsoft/Phi-3-mini-4k-instruct#hardware
    # attn_implementation="eager", # remove on newer hardware
    trust_remote_code=True
)

config.json:   0%|          | 0.00/4.53k [00:00<?, ?B/s]

configuration_phimoe.py:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-MoE-instruct:
- configuration_phimoe.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phimoe.py:   0%|          | 0.00/80.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-MoE-instruct:
- modeling_phimoe.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00016-of-00017.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00017-of-00017.safetensors:   0%|          | 0.00/3.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

CPU times: user 3min 46s, sys: 3min 5s, total: 6min 52s
Wall time: 23min 11s


In [14]:
!nvidia-smi

Fri Aug 23 07:40:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              52W / 400W |  25473MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [15]:
positive = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [16]:
negative = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [34]:
assessment = negative[0]
# assessment = positive[0]

In [35]:
%%time

messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a very brief justification for your assessment.

# Assessment
{assessment}

'''},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

CPU times: user 6.62 s, sys: 0 ns, total: 6.62 s
Wall time: 6.6 s


No, a positive recommendation is not given. Justification: The assessment indicates a lack of sufficient evidence from the diagnosis to support a regulatory decision.

In [41]:
def eval_assessment(assessment):
  messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a very brief justification for your assessment.

# Assessment
{assessment}

'''},
]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith("Yes"):
    return "Positive", result
  elif result.startswith("No"):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [42]:
%%time

negative_results = []
negative_explanations = []

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  negative_results.append(result)
  negative_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: No specific findings can be derived from the diagnosis currently named as the basis for the regulation.
Negative: No, a positive recommendation is not given. Justification: The assessment indicates a lack of sufficient evidence from the diagnosis to support a regulatory decision.
-----
Assessment: According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.
Positive: Yes, a positive recommendation is given. Justification: The insured has received the requested functional product within its area of application, indicating compliance with the service provision. Yes, a positive recommendation is given. Justification: The provision of the requested functional product to the insured within its designated area of application demonstrates adherence to the required service standards. Yes, a positive recommendation is given. Justification: The insured's receipt of the re

## Positive

In [43]:
%%time

positive_results = []
positive_explanations = []

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  positive_results.append(result)
  positive_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")


Assessment: With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.
Positive: Yes, a positive recommendation is given. Justification: The assessment acknowledges the conceivable need for compensation to meet basic needs, indicating a positive stance towards providing support.
-----
Assessment: The socio-medical prerequisites for the prescribed aid supply have been met.
Positive: Yes, a positive recommendation is given. The assessment confirms that the socio-medical prerequisites for the prescribed aid supply have been met.
-----
Assessment: Everyday relevant usage benefits have been determined.
Positive: Yes, a positive recommendation is given. Justification: The assessment explicitly states that "everyday relevant usage benefits" have been determined, indicating a favorable outcome. Yes, a positive recommendation is given. Justification: The assessment explicitly states that "everyday relevant usage benefits" have been determined, indicating a

In [44]:
!nvidia-smi

Fri Aug 23 08:05:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              60W / 400W |  25865MiB / 40960MiB |     24%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [46]:
import pandas as pd

df = pd.DataFrame({
    'assesment': negative + positive,
    'y_true': ['Negative'] * len(negative) + ['Positive'] * len(positive),
    'y_hat': negative_results + positive_results,
    'explanation': negative_explanations + positive_explanations
})
df

,assesment,y_true,y_hat,explanation
0,No specific findings can be derived from the d...,Negative,Negative,"No, a positive recommendation is not given. Ju..."
1,According to the service extracts from the hea...,Negative,Positive,"Yes, a positive recommendation is given. Justi..."
2,A medically comprehensible explanation as to w...,Negative,Negative,"No, a positive recommendation is not given. Th..."
3,From an overall view of the information availa...,Negative,Negative,"No, a positive recommendation is not given. Th..."
4,A medical justification for why a product not ...,Negative,Negative,"No, a positive recommendation is not given. Ju..."
5,"With the diagnosis named here, the need for co...",Positive,Positive,"Yes, a positive recommendation is given. Justi..."
6,The socio-medical prerequisites for the prescr...,Positive,Positive,"Yes, a positive recommendation is given. The a..."
7,Everyday relevant usage benefits have been det...,Positive,Positive,"Yes, a positive recommendation is given. Justi..."
8,Socio-medical indication for the aid is confir...,Positive,Positive,"Yes, a positive recommendation is given. The s..."
9,Contraindications have been excluded; there ar...,Positive,Positive,"Yes, a positive recommendation is given. Justi..."


In [49]:
df.to_excel(f'results-{kind}.xlsx', index=False)